In [20]:
import traitlets
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
car_blocked_slider = widgets.FloatSlider(description='car_blocked', min=0.0, max=1.0, orientation='vertical')
caution_blocked_slider = widgets.FloatSlider(description='caution_blocked', min=0.0, max=1.0, orientation='vertical')
limit_blocked_slider = widgets.FloatSlider(description='limit_blocked', min=0.0, max=1.0, orientation='vertical')
pedestrian_blocked_slider = widgets.FloatSlider(description='pedestrian_blocked', min=0.0, max=1.0, orientation='vertical')
scooter_blocked_slider = widgets.FloatSlider(description='scooter_blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, car_blocked_slider, caution_blocked_slider, limit_blocked_slider, pedestrian_blocked_slider, scooter_blocked_slider]))

In [1]:
from jetbot import Robot
robot = Robot()

In [2]:
import torch
import torchvision

road = torchvision.models.resnet18(pretrained=False)
road.fc = torch.nn.Linear(512, 2)

In [3]:
road.load_state_dict(torch.load('5000_steering_model_xy.pth'))

<All keys matched successfully>

In [4]:
car = torchvision.models.alexnet(pretrained=False)
car.classifier[6] = torch.nn.Linear(car.classifier[6].in_features, 2)

In [5]:
car.load_state_dict(torch.load('car_model.pth'))

<All keys matched successfully>

In [6]:
caution = torchvision.models.alexnet(pretrained=False)
caution.classifier[6] = torch.nn.Linear(caution.classifier[6].in_features, 2)

In [7]:
caution.load_state_dict(torch.load('caution_model.pth'))

<All keys matched successfully>

In [8]:
limit = torchvision.models.alexnet(pretrained=False)
limit.classifier[6] = torch.nn.Linear(limit.classifier[6].in_features, 2)

In [9]:
limit.load_state_dict(torch.load('limit_model.pth'))

<All keys matched successfully>

In [10]:
pedestrian = torchvision.models.alexnet(pretrained=False)
pedestrian.classifier[6] = torch.nn.Linear(pedestrian.classifier[6].in_features, 2)

In [11]:
pedestrian.load_state_dict(torch.load('pedestrian_model.pth'))

<All keys matched successfully>

In [12]:
scooter = torchvision.models.alexnet(pretrained=False)
scooter.classifier[6] = torch.nn.Linear(scooter.classifier[6].in_features, 2)

In [13]:
scooter.load_state_dict(torch.load('scooter_model.pth'))

<All keys matched successfully>

In [14]:
road_device = torch.device('cuda')
road = road.to(road_device)
road = road.eval().half()

In [15]:
car_device = torch.device('cuda')
car = car.to(car_device)

In [16]:
caution_device = torch.device('cuda')
caution = caution.to(caution_device)

In [17]:
limit_device = torch.device('cuda')
limit = limit.to(limit_device)

In [18]:
pedestrian_device = torch.device('cuda')
pedestrian = pedestrian.to(pedestrian_device)

In [19]:
scooter_device = torch.device('cuda')
scooter = scooter.to(scooter_device)

In [21]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import time

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

mean1 = 255.0 * np.array([0.485, 0.456, 0.406])
stdev1 = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean1, stdev1)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(road_device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def preprocess1(camera_value):
    global car_device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(car_device)
    x = x[None, ...]
    return x

def preprocess2(camera_value):
    global caution_device
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(caution_device)
    x = x[None, ...]
    return x

def preprocess3(camera_value):
    global limit_device
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(limit_device)
    x = x[None, ...]
    return x

def preprocess4(camera_value):
    global pedestrian_device
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(pedestrian_device)
    x = x[None, ...]
    return x

def preprocess5(camera_value):
    global scooter_device
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(scooter_device)
    x = x[None, ...]
    return x

In [22]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [23]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last, car_blocked_slider, caution_blocked_slider, limit_blocked_slider, pedestrian_blocked_slider, scooter_blocked_slider, robot
    image = change['new']

    x0 = preprocess1(image)
    y0 = car(x0)
    x1 = preprocess2(image)
    y1 = caution(x1)
    x2 = preprocess3(image)
    y2 = limit(x2)
    x3 = preprocess4(image)
    y3 = pedestrian(x3)
    x4 = preprocess5(image)
    y4 = scooter(x4)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y0 = F.softmax(y0, dim=1)
    y1 = F.softmax(y1, dim=1)
    y2 = F.softmax(y2, dim=1)
    y3 = F.softmax(y3, dim=1)
    y4 = F.softmax(y4, dim=1)
    
    car_blocked = float(y0.flatten()[0])
    caution_blocked = float(y1.flatten()[0])
    limit_blocked = float(y2.flatten()[0])
    pedestrian_blocked = float(y3.flatten()[0])
    scooter_blocked = float(y4.flatten()[0])
    
    car_blocked_slider.value = car_blocked
    caution_blocked_slider.value = caution_blocked
    limit_blocked_slider.value = limit_blocked
    pedestrian_blocked_slider.value = pedestrian_blocked
    scooter_blocked_slider.value = scooter_blocked
    
    xy = road(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    if car_blocked > 0.9 or caution_blocked > 0.9 or limit_blocked > 0.9 or pedestrian_blocked > 0.9 or scooter_blocked > 0.9:
        robot.stop()
        
    else:
        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
    time.sleep(0.001)
    
execute({'new': camera.value})

In [ ]:
camera.observe(execute, names='value')

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

In [ ]:
camera.stop()